# Chapter 2: Information Criterions and Bootstrap Confidence Intervals

In this chapter, we'll cover the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC) for nested models, as well as the bootstrap method for confidence intervals. 

## Nested Models

Let's say we have model A with variables $\theta_1$, $\theta_2$ and $\theta_3$. Model B is nested in model A if model B contains a subset of the variables of model A, e.g. if model B has variables $\theta_1$ and $\theta_3$. We call model B a $\textbf{nested model}$. 
Model B is $\textbf{not}$ a nested model if it contains a variable that model A doesn't contain, e.g. if model B has variables $\theta_1$, $\theta_3$ and $\theta_4$. 

[overgang van nested models naar AIC and BIC en waarom we naar nested models kijken]


## Akaike Information Criterion (AIC)

## Bayesian Information Criterion (BIC)


tekst voor AIC en BIC

Zo krijg je een formule in de tekst
$AIC = 2*2=4$

Zo krijg je een formule in het midden. De tag is voor de nummering.
<a id="eq:AIC"></a>
$$AIC = 2k -\ln(L),\tag{1}$$
De unit boven [Eq. 1](#eq:AIC) maakt het mogelijk om er zo naar te refereren.



## Bootstrap

tekst voor bootstrap voor testen hihi